In [1]:
import datetime as dt
from libs.utils.params import Params

params = Params(
    size=1000,
    # Random state
    seed=None,
    # Thresholds
    threshold=Params(
        adaptative=True,
        opt=0.9,
        initial=0.9,
        min=0.6,
        step=0.05,
        expressive=0.5,
        current=0.9,
    ),
    max_depth=4,
    max_depth_step=0,
    patterns=Params(
        individuals=True,
        existential=True,
    ),
    embeddings="toy",
    clustering=Params(
        affinity="euclidean",
        linkage="ward",
    ),
    metric="harmonic",
    max_axioms=2,
    min_gain=0.08,
    allow_child=False,
    sort_axioms=False,
    others=Params(
        keep=True,
        n=8, # Max number of candidates to keep
        threshold=0.9, # % of the optimal score
    ),
    halting=Params(
        min_size=30,
        max_rec_steps=40,
        max_clustering_steps=100,
        max_extracted_depth=15,
        memory_limit=110*1024**2 # gigabytes
    ),
    extra=Params(
        active=True,
        n=100,
        reset_classes=True,
        depth=20,
        threshold=0.15
    ),
    record=Params(
        save_taxonomy=True,
        checkpoints=True,
        checkpoint_every=100,
        dirname="results/taxonomy/auto",
        name_pattern="taxonomy_{halting.max_clustering_steps}s_{timestamp:%m%d_%Hh%M}"
    ),
    display=True
)
#params.record.taxname = params.record.name_pattern.format(**params)

from libs.expressive import BASE_PARAMS as params

params.save("test.json")

In [1]:
from libs.graph import KnowledgeGraph

kg = KnowledgeGraph.from_dir("toy")

Triples: 100%|█████████████████████████████████████████████████████████████| 316114/316114 [00:02<00:00, 105589.71it/s]


In [2]:
kg.to_dir("data/graph/toy2")

In [38]:
%autoreload

import libs.expressive.extractor as exp
from importlib import reload
import libs.sampling as lsamp
from libs.axiom import Existential, Concept

reload(lsamp)
reload(exp)


relevant_ids = {h for h, rs in kg._h.items() if len(rs) > 3}
sampler = lsamp.NaiveGraphSampler(kg, relevant_ids)

# Run 10 clustering steps
extr = exp.ExpressiveExtractor(kg, params, sampler=sampler, verbose="INFO")
extr.init()
extr.run(10)

# Print the resulting taxonomy
T = extr.get_taxonomy()
T.print()

Directory 'results/taxonomy/auto\taxonomy_100s_0917_14h00' already exists.
Override ? y/[n]y
14:00:15 - INFO : Initialisation done.
14:00:15 - INFO : Initialisation done.
14:00:15 - INFO : STEP 0: starting with axiom ⊤
14:00:15 - INFO : STEP 0: starting with axiom ⊤
14:00:16 - INFO : Subclasses found: ∃foaf:givenName.{<LABEL:en>}, ∃dbo:isPartOf.dbo:Settlement
14:00:16 - INFO : Subclasses found: ∃foaf:givenName.{<LABEL:en>}, ∃dbo:isPartOf.dbo:Settlement
14:00:16 - INFO : STEP 1: starting with axiom ∃foaf:givenName.{<LABEL:en>}
14:00:16 - INFO : STEP 1: starting with axiom ∃foaf:givenName.{<LABEL:en>}
14:00:16 - INFO : STEP 1: starting with axiom ∃dbo:isPartOf.dbo:Settlement
14:00:16 - INFO : STEP 1: starting with axiom ∃dbo:isPartOf.dbo:Settlement
14:00:17 - INFO : Subclasses found: ∃dbo:country.{dbr:Iran}, ∃dbo:type.{dbr:Village}
14:00:17 - INFO : Subclasses found: ∃dbo:country.{dbr:Iran}, ∃dbo:type.{dbr:Village}
14:00:17 - INFO : STEP 2: starting with axiom ∃dbo:country.{dbr:Iran}
14:

In [33]:
%autoreload
from libs.axiom import Existential, Concept, RemainderAxiom, REM, TopAxiom
from libs.sampling import NaiveGraphSampler

a = Concept("dbo:PopulatedPlace")
b = Concept(singleton="<STRING>")
c = Existential("dbo:postalCode", b)
d = Existential("dbo:country", a)
e = RemainderAxiom(TopAxiom & a, c)

axioms = [a, b, c, d, e, TopAxiom]

print(*axioms)

sampler = NaiveGraphSampler(kg)
print("Instances found:")
for axiom in axioms:
    i, n = sampler.sample(axiom, 10)
    print(f" - {n} for axiom {axiom}")

dbo:PopulatedPlace {<STRING>} ∃dbo:postalCode.{<STRING>} ∃dbo:country.dbo:PopulatedPlace *(⊤∧dbo:PopulatedPlace) ⊤
Instances found:
 - 12541 for axiom dbo:PopulatedPlace
 - 0 for axiom {<STRING>}
 - 0 for axiom ∃dbo:postalCode.{<STRING>}
 - 5028 for axiom ∃dbo:country.dbo:PopulatedPlace
 - 12541 for axiom *(⊤∧dbo:PopulatedPlace)
 - 47694 for axiom ⊤


In [32]:
f =  TopAxiom & a

**Sampling Examples**

For now, `GraphSampler` is not working properly so we use `NaiveGraphSampler`. In production, `NaiveGraphSampler` should not be used since it performs a full scan of the knowledge graph and thus is inefficient for real-world, large-scale graphs.

In [8]:
import libs.sampling as lsamp
from importlib import reload
reload(lsamp)
from libs.axiom import Existential, Concept

# An example of complex axiom (arity 3)
axiom = Existential("dbo:nationality", Concept(singleton="dbr:Japan")) & Concept("dbo:Agent")
print(axiom)

# Only consider entities involved in 4+ relations
relevant_ids = {h for h, rs in kg._h.items() if len(rs) > 3}
sampler = lsamp.NaiveGraphSampler(kg, relevant_ids)

# Sample 10 entities from this axiom
instances, size = sampler.sample(axiom, 10)
kg.ent.to_uris(*instances)

∃dbo:nationality.{dbr:Japan}∧dbo:Agent


['dbr:Yasunori_Nomura',
 'dbr:Kagami_Yoshimizu',
 'dbr:Tateo_Ozaki',
 'dbr:Hikari_Okubo',
 'dbr:Nabi_Tajima',
 'dbr:Saeko_Kimura',
 'dbr:Masahiko_Amakasu']

**Axiom Inducer Module**

This is a demo of the class `libs.axiom_extraction.Inducer`. For inducing axioms, we need a set of positive examples $E^+$,  a set of negative examples $E^-$ and a threshold $\delta \in [0, 1]$. In this demo, $E^+$ is a random subset of class `dbo:Athlete`, $E^-$ is a random subset of class `dbo:City`, and $\delta=0.85$.

The algorithm outputs a list of axioms that have a partition score higher than $\delta$.

In [7]:
import libs.axiom_extraction as lae
from importlib import reload
import random
lae = reload(lae)


size = 5
delta = 0.85

samples = []
for cls in ("dbo:Athlete", "dbo:City"):
    clsid = graph.ent.to_id(cls)
    instances = [x for x in relevant_ids if (x, kg.isaid, clsid) in kg]
    if verbose and size > len(instances):
        print(f"WARNING: Can't sample {size} items out of {len(instances)} instances of class {cls}")
    size = min(size, len(instances))

    samples.append(random.sample(instances, size))

E_pos, E_neg = samples
ind = lae.Inducer(E_pos, E_neg, kg, threshold=0.0, verbose=True)

print(ind)
res = ind.find(allow_neg=False, threshold=delta)
res

Inducer(entities=10, axioms=125)
Finding axioms

Step 0/3: 1 axioms to improve
Improving __empty__...
Coverage too low (0.00<0.85). Adding OR clauses...
Specificity too low (0.00<0.85). Adding AND clauses...
...250 results found

Step 1/3: 5 axioms to improve
Improving ∃dbo:isPartOf.dbo:AdministrativeRegion...
Coverage too low (0.40<0.85). Adding OR clauses...
...124 results found
Improving ∃dbo:isPartOf.{dbr:Una_district}...
Coverage too low (0.20<0.85). Adding OR clauses...
...124 results found
Improving ∃dbo:part.{dbr:Human_settlement}...
Coverage too low (0.20<0.85). Adding OR clauses...
...124 results found
Improving ∃dbo:position.{dbr:Attacking_Midfielder}...
Coverage too low (0.00<0.85). Adding OR clauses...
Specificity too low (0.80<0.85). Adding AND clauses...
...248 results found
Improving ∃dbo:nationality.dbo:Place...
Coverage too low (0.00<0.85). Adding OR clauses...
Specificity too low (0.60<0.85). Adding AND clauses...
...248 results found

Step 2/3: 5 axioms to improve
I

axiom,cov,spe,sco,
0,,,,
∃dbo:isPartOf.dbo:AdministrativeRegion,0.40,1.00,0.70,0
∃dbo:isPartOf.{dbr:Una_district},0.20,1.00,0.60,0
∃dbo:part.{dbr:Human_settlement},0.20,1.00,0.60,0
∃dbo:position.{dbr:Attacking_Midfielder},0.00,0.80,0.40,0
∃dbo:nationality.dbo:Place,0.00,0.60,0.30,0
1,,,,
∃dbo:isPartOf.dbo:AdministrativeRegion∨∃dbo:isPartOf.{dbr:Una_district},0.60,1.00,0.80,1
∃dbo:isPartOf.dbo:AdministrativeRegion∨∃dbo:part.{dbr:Human_settlement},0.60,1.00,0.80,1
∃dbo:isPartOf.{dbr:Una_district}∨∃dbo:isPartOf.dbo:AdministrativeRegion,0.60,1.00,0.80,1
